In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


#### Pipeline Steps:

    Getting access to the webcam.
    Setup mouse event.
    Initialize the tracker.
    Track an object in real-time.

### Step-1 : Getting access to the webcamp

In [4]:
# set a name for the window
windowName= 'Output Window'
cv2.namedWindow(windowName)

#Initialize a captureVideo object
cap= cv2.VideoCapture(0)

while True:
    
    #reads each frame from the webcam
    ret, frame= cap.read()
    output= frame
    
    #show frame in new window
    cv2.imshow(windowName, output)
    
    #waits 1 ms between each frame and continues to show the frames untill pressed 'q'
    if cv2.waitKey(1) == ord('q'):
        break

#release the capture object
cap.release()

#close all active windows
cv2.destroyAllWindows()

## Step 2: Mouse Event

Why are we using mouse events in this project?

Here, we don’t want our tracker to track only one specific object at each program run. But we want to select in real-time to track an object and can also change the track focus to another object without closing the program.

In [26]:
x, y, w, h= 0, 0, 0, 0
first_point_saved= False
second_point_saved= False
track_window= (x, y, h, w)
can_track= False

# this function returns the coordinate points of the opencv window where the event happens
def click_event(event, px, py):
    global x, y, w, h, first_point_saved, second_point_saved, track_window, can_track
    
    # left mouse button released
    if event== cv2.EVENT_LBUTTONUP:
        
        print(cv2.EVENT_LBUTTONUP)
        
        #if true means first point is saved then we calculate the tracker window height and width.
        if first_point_saved:
            
            w= px-x
            h= py-y
            
            track_window= (x, y, w, h)
            print(x, y, w, h)
            
            first_point_saved= False
            second_point_saved= True
            
            
        else: 
            x= px
            y= py
            
            first_point_saved= True
            can_track= False
        
    # this will stop the tracker from tracking
    if event== cv2.EVENT_RBUTTONDOWN:
        can_track= False

cv2.setMouseCallback(windowName, click_event)       #start event mouse

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2543: error: (-27:Null pointer) NULL window: 'Output Window' in function 'cvSetMouseCallback'


In [16]:
click_event( cv2.EVENT_LBUTTONUP, 6, 5)

4
6 5 0 0


###  Step 3: Initialize the Tracker

In [27]:
#initialize tracker
def initializeTracker(frame, track_window):
    x, y, w, h= track_window
    
    # Camshift is a colorbased tracker so we first set up the ROI. 
    # ROI is basically the cropped image of the selcted object's window.
    
    roi= frame[y:y+h, x:x+w]
    
    hsv_roi= cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    # Calculate the histogram of the colors
    roi_hist= cv2.calcHist([hsv_roi],[0], None, [180], [0,180])
    
    # filters some noises in the histogram.
    roi_hist= cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
    
    return roi_hist, roi

# value for the criteria 10 and 1 means the tracker updates either every 10 iterations or moves by at least 1 point.
term_crit= (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

    

### Step 4: Track an object in real time

In [24]:
# check if 2nd point is also saved and then initialize the tracker according to the selected object.

hsv= cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

if second_point_saved:
    
    roi_hist, roi= initializeTracker(frame, windowName)
    second_point_saved= False
    can_track= True
    
# start tracking
# If can_track is false but first_point is saved then it draws a circle wherever the mouse release event happens.
if can_track:
    
    # finds the same color distribution of the ROI’s histogram in the current frame.
    dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
    
    # apply camshift to get the new location
    ret, track_window= cv2.CamShift(dst, track_window, term_crit)
    
    # After that call the cv2.CamShift() method to track the selected object. The method takes the Back projected histogram, track window (selected object’s window), 
    # and the term criteria as argument and returns the objects box information and updated track window
    pts= cv2.boxPoints(ret)
    pts= np.int0(pts)
    print(ret)
    
    cv2.imshow('roi', roi)
    output= cv2.polylines(frame, [pts], True, 255, 2)
    
else:
    output= frame
    if first_point_saved:
        cv2.circle(output, (x,y), 5, (0,0,255), -1)
        cv2.destroyAllWindows('roi')

cv2.imshow(windowName, output)


ValueError: too many values to unpack (expected 4)